En este archivo, voy a desarrollar la función que se me pide en el cuarto endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_4.png "Title")

# Importaciones necesarias para trabajar

In [1]:
import pandas as pd

Archivos que voy a usar para desarrollar este endpoit

In [2]:
reseñas = pd.read_parquet("../EDA/user_reviews_complete.parquet")
reseñas.head(2)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479


In [3]:
juegos = pd.read_parquet("../Datasets/steam_games_complete.parquet")
juegos.head(2)

,item_id,item_name,developer,genres,tags,specs,release_date,price
88310,761140,Lost Summoner Kitty,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99
88311,643980,Ironbound,Secret Level SRL,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,0.00


Repasemos un poco el concepto de la función.

Ingresando un año a la misma, nos tiene que devolver un top 3 de desarrolladores con más juegos recomendados en el año ingresado.


En nuestros dataset, tenemos dos variables de las cuáles podemos obtener una fecha. Por un lado, tenemos la columna release_date del dataset de juegos, y por otro lado, tenemos posted del dataset de reseñas. 
Analizando lo que se nos pide en la función, "devuelve un top 3 desarrolladores con más juegos recomendados en el año ingresado", noto que hace referencia a la última columna mencionada, "posted", ya que la columna "release_date" hace referencia a la fecha de salida del videojuego, dónde no necesariamente fué escrita la reseña, sobre todo teniendo en consideración aquellos juegos que salieron antes de que Steam fuese creado. 

Hechemos un vistazo ahora a las columnas que tenemos disponibles en ambos datasets.

In [4]:
print(juegos.columns)
print()
print(reseñas.columns)

Index(['item_id', 'item_name', 'developer', 'genres', 'tags', 'specs',
       'release_date', 'price'],
      dtype='object')

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id'],
      dtype='object')


Del primer dataset, 'item_id' y 'developer' son las únicas columnas relevantes para el desarrollo de nuestro endpoint, ya que el resto no nos aportan información significativa. Con "item_id" puedo relacionar ambas tablas, y con "developer" puedo determinar el desarrollador en cuestión que obtuvo determinada cantidad de reseñas. 

Del segundo dataset, 'posted', 'item_id', 'recommend' y 'review' son las únicas columnas relevantes. Con "posted" determino la fecha en la que fué escrita la reseña, con "item_id" relaciono ambas tablas, con "recommend" determino si es True o False y con "review" si es un comentario positivo, neutro o negativo.

Procedo a eliminar las columnas que no me sirven.

In [5]:
for column in juegos.columns:
    if column not in ['item_id', 'developer']:
        juegos.drop(column, axis=1, inplace=True)
print(juegos.columns)

for column in reseñas.columns:
    if column not in ['posted', 'item_id', 'recommend', 'review']:
        reseñas.drop(column, axis=1, inplace=True)
print(reseñas.columns)

Index(['item_id', 'developer'], dtype='object')
Index(['posted', 'item_id', 'recommend', 'review'], dtype='object')


Una vez eliminadas, echemos un vistazo a como quedaron las tablas.

In [6]:
juegos

,item_id,developer
88310,761140,Kotoshiro
88311,643980,Secret Level SRL
88312,670290,Poolians.com
88313,767400,彼岸领域
88315,772540,Trickjump Games Ltd
...,...,...
120439,745400,Bidoniera Games
120440,773640,"Nikita ""Ghost_RUS"""
120441,733530,Sacada
120442,610660,Laush Dmitriy Sergeevich


In [7]:
reseñas

,posted,item_id,recommend,review
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59300,Posted July 10.,70,True,a must have classic from steam definitely wort...
59301,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59302,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59303,Posted July 20.,730,True,:D


Lo que tendría que hacer a continuación es normalizar la columna "posted", para tener sólo los años de las reseñas. Lo hago a continuación.

In [8]:
# Función para extraer el año
def extraer_año(texto):
    import re
    match = re.search(r'\b\d{4}\b', texto)
    return match.group(0) if match else None

reseñas["año"] = reseñas["posted"].apply(extraer_año)
reseñas

,posted,item_id,recommend,review,año
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2011
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2011
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2014
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
59300,Posted July 10.,70,True,a must have classic from steam definitely wort...,None
59301,Posted July 8.,362890,True,this game is a perfect remake of the original ...,None
59302,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,None
59303,Posted July 20.,730,True,:D,None


Voy a chequear cuantos nulos me quedaron en esta última columna creada

In [9]:
reseñas.isnull().sum()

posted           0
item_id          0
recommend        0
review           0
año          10119
dtype: int64

In [10]:
reseñas[reseñas["año"].isnull()]

,posted,item_id,recommend,review,año
6,Posted February 3.,248820,True,A suitably punishing roguelike platformer. Wi...,None
27,Posted May 20.,730,True,ZIKA DO BAILE,None
28,Posted July 24.,730,True,BEST GAME IN THE BLOODY WORLD,None
31,Posted June 16.,252950,True,love it,None
32,Posted June 11.,440,True,mt bom,None
...,...,...,...,...,...
59300,Posted July 10.,70,True,a must have classic from steam definitely wort...,None
59301,Posted July 8.,362890,True,this game is a perfect remake of the original ...,None
59302,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,None
59303,Posted July 20.,730,True,:D,None


In [11]:
reseñas[(reseñas["año"].isnull()) & (reseñas["recommend"] == True)]["posted"]

6        Posted February 3.
27           Posted May 20.
28          Posted July 24.
31          Posted June 16.
32          Posted June 11.
                ...        
59300       Posted July 10.
59301        Posted July 8.
59302        Posted July 3.
59303       Posted July 20.
59304        Posted July 2.
Name: posted, Length: 8291, dtype: object

Es una lástima, por que son un 17% de las filas las que no tienen la información más relevante acerca de la fecha (el año), y que por lo tanto, debo borrar al no saber eso mismo. 

In [12]:
reseñas = reseñas[~reseñas["año"].isnull()]
reseñas

,posted,item_id,recommend,review,año
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2011
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2011
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2014
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
59252,"Posted October 14, 2015.",730,True,its FUNNNNNNNN,2015
59255,"Posted October 10, 2015.",253980,True,Awesome fantasy game if you don't mind the gra...,2015
59265,"Posted October 31, 2015.",730,True,Prettyy Mad Game,2015
59267,"Posted December 14, 2015.",730,True,AMAZING GAME 10/10,2015


Al ya haber extraído la información importante de la misma, la columna "posted" no tiene mayor relevancia, por lo cuál, decido eliminarla.

In [13]:
reseñas.drop("posted",axis=1,inplace=True)
reseñas

C:\Users\crisr\AppData\Local\Temp\ipykernel_16248\657766983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reseñas.drop("posted",axis=1,inplace=True)


,item_id,recommend,review,año
0,1250,True,Simple yet with great replayability. In my opi...,2011
1,22200,True,It's unique and worth a playthrough.,2011
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,251610,True,I know what you think when you see this title ...,2014
4,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...
59252,730,True,its FUNNNNNNNN,2015
59255,253980,True,Awesome fantasy game if you don't mind the gra...,2015
59265,730,True,Prettyy Mad Game,2015
59267,730,True,AMAZING GAME 10/10,2015


In [14]:
reseñas["año"]

0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
59252    2015
59255    2015
59265    2015
59267    2015
59276    2015
Name: año, Length: 49186, dtype: object

Cambio el tipo de dato, que me quedó en object, pero que tiene que ser int para el correcto funcionamiento de la función

In [15]:
reseñas["año"] = reseñas["año"].astype(int)
reseñas["año"]

C:\Users\crisr\AppData\Local\Temp\ipykernel_16248\279902882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reseñas["año"] = reseñas["año"].astype(int)


0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
59252    2015
59255    2015
59265    2015
59267    2015
59276    2015
Name: año, Length: 49186, dtype: int32

Primer prototipo del endpoint

In [16]:
def mejor_desarrollador_del_año(año): # En este primer prototipo, voy a ingresar un año a la función, y me va a devolver las reseñas escritas en ese año.
    try:
        año_int = int(año)
        reseñas_del_año_dado = reseñas[reseñas["año"] == año_int]
        return reseñas_del_año_dado
    except Exception as e:
        return e

In [17]:
mejor_desarrollador_del_año(2012) # La salida es la esperada

,item_id,recommend,review,año
13,20920,True,"Really Really Really Great Game, very good sto...",2012
14,204100,True,"Just buy it already. Great Story, Great Multip...",2012
16,207610,True,The ending to this game is.... ♥♥♥♥♥♥♥.... Jus...,2012
17,108710,True,"Alan wake is a really good game, the light eff...",2012
23,207610,True,"It reminds me of that TV Show called ""The Walk...",2012
...,...,...,...,...
53289,440,True,░░░░░░░░░░░░▄▄░░░░░░░░░░░█░░█░░░░░░░░░░░█░░█░░...,2012
53571,108800,True,This is the best game i have right now! Awesom...,2012
53787,49520,True,"its amazingly epic, with cookies :D",2012
54209,440,True,DOWNLOAD IT,2012


In [18]:
mejor_desarrollador_del_año(2013) # Nuevamente la salida es la esperada

,item_id,recommend,review,año
4,227300,True,For a simple (it's actually not all that simpl...,2013
5,239030,True,Very fun little game to play when your bored o...,2013
12,250320,True,This game... is so fun. The fight sequences ha...,2013
19,211820,True,"It's like Terraria, you play for 9 hours strai...",2013
21,204300,True,"OH YES, THIS GAME IS THE BEST, THEY ADD STUFF ...",2013
...,...,...,...,...
58339,239350,True,In this game there are sixteen levels.The goal...,2013
58340,225080,True,:D,2013
58341,550,True,/,2013
58342,105600,True,",",2013


Segundo prototipo del endpoint

In [19]:
def mejor_desarrollador_del_año(año): # En este segundo prototipo, voy a ingresar un año a la función, y me va a devolver las reseñas escritas en ese año que tengan la condición de recommend = True.
    try:
        año_int = int(año)
        reseñas_del_año_dado = reseñas[(reseñas["año"] == año_int) & (reseñas["recommend"] == True)]
        return reseñas_del_año_dado
    except Exception as e:
        return e

In [20]:
mejor_desarrollador_del_año(2012) # Me devuelve menor cantidad de filas que anteriormente con el mismo año, así que concluyo que la salida es la esperada.

,item_id,recommend,review,año
13,20920,True,"Really Really Really Great Game, very good sto...",2012
14,204100,True,"Just buy it already. Great Story, Great Multip...",2012
16,207610,True,The ending to this game is.... ♥♥♥♥♥♥♥.... Jus...,2012
17,108710,True,"Alan wake is a really good game, the light eff...",2012
23,207610,True,"It reminds me of that TV Show called ""The Walk...",2012
...,...,...,...,...
53289,440,True,░░░░░░░░░░░░▄▄░░░░░░░░░░░█░░█░░░░░░░░░░░█░░█░░...,2012
53571,108800,True,This is the best game i have right now! Awesom...,2012
53787,49520,True,"its amazingly epic, with cookies :D",2012
54209,440,True,DOWNLOAD IT,2012


In [21]:
mejor_desarrollador_del_año(2013) # Lo mismo sucede con 2013.

,item_id,recommend,review,año
4,227300,True,For a simple (it's actually not all that simpl...,2013
5,239030,True,Very fun little game to play when your bored o...,2013
12,250320,True,This game... is so fun. The fight sequences ha...,2013
19,211820,True,"It's like Terraria, you play for 9 hours strai...",2013
21,204300,True,"OH YES, THIS GAME IS THE BEST, THEY ADD STUFF ...",2013
...,...,...,...,...
58339,239350,True,In this game there are sixteen levels.The goal...,2013
58340,225080,True,:D,2013
58341,550,True,/,2013
58342,105600,True,",",2013


Tercer prototipo del endpoint

In [22]:
def mejor_desarrollador_del_año(año):
    """
    En este tercer prototipo, voy a ingresar un año a la función, y me va a devolver cuantas reseñas son por cada juego escritas en ese año y que además tengan la condición de recommend = True.
    """

    try:
        año_int = int(año)
        reseñas_del_año_dado = reseñas[(reseñas["año"] == año_int) & (reseñas["recommend"] == True)]
        reseñas_positivas_por_juego = reseñas_del_año_dado["item_id"].value_counts()
        return reseñas_positivas_por_juego
    except Exception as e:
        return e

In [23]:
año_2013 = mejor_desarrollador_del_año(2013)

Cuarto prototipo del endpoint

In [24]:
def mejor_desarrollador_del_año(año):
    """
    En este cuarto prototipo, voy a ingresar un año a la función, y me va a devolver una lista de los top 7 desarrolladores
    """

    try:
        lista_retorno = []
        año_int = int(año)
        reseñas_del_año_dado = reseñas[(reseñas["año"] == año_int) & (reseñas["recommend"] == True)]
        reseñas_positivas_por_juego = reseñas_del_año_dado["item_id"].value_counts()
        contador = 0
        for id_juego in reseñas_positivas_por_juego.index:
            try:
                desarrollador = juegos[juegos["item_id"] == id_juego]["developer"].iloc[0]
                lista_retorno.append(desarrollador)
                contador += 1
                if contador == 7:
                    break
            except Exception as e:
                print(e)
        return lista_retorno
    except Exception as e:
        return e
    
mejor_desarrollador_del_año(2013)

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds


['Valve',
 'Facepunch Studios',
 'Valve',
 'Valve',
 'Re-Logic',
 'Chucklefish',
 'Digital Extremes']

Como podemos observar, Valve se queda con el podio, y un detalle importante a considerar también es que aparece varias veces en el top 7, que es justamente el caso con el que me quería encontrar para optimizar bien la función. Seguiremos usando el año 2013 cómo ejemplo

Quinto prototipo del endpoint

In [25]:
def mejor_desarrollador_del_año(año):
    """
    En este quinto prototipo, voy a ingresar un año a la función, y me va a devolver una lista de los top 3 desarrolladores sin repeticiones
    """

    try:
        lista_retorno = []
        año_int = int(año)
        reseñas_del_año_dado = reseñas[(reseñas["año"] == año_int) & (reseñas["recommend"] == True)]
        reseñas_positivas_por_juego = reseñas_del_año_dado["item_id"].value_counts()
        desarrolladores = 0
        for id_juego in reseñas_positivas_por_juego.index:
            try:
                desarrollador = juegos[juegos["item_id"] == id_juego]["developer"].iloc[0]
                lista_retorno.append(desarrollador)
                desarrolladores += 1
                if desarrolladores == 10: # Por las dudas dejo el "top 10" desarrolladores, aunque en realidad me voy a quedar con los primeros 3 unicos.
                    break
            except Exception as e:
                continue
        lista_desarrolladores = list(dict.fromkeys(lista_retorno))

        return lista_desarrolladores[:3]
    except Exception as e:
        return e
    
mejor_desarrollador_del_año(2013)

['Valve', 'Facepunch Studios', 'Re-Logic']

Sexto prototipo del endpoint

In [26]:
def mejor_desarrollador_del_año(año):
    """
    En este sexto prototipo, voy a ingresar un año a la función, y me va a devolver una lista de diccionarios con los top 3 desarrolladores
    """

    try:
        lista_retorno = []
        año_int = int(año)
        reseñas_del_año_dado = reseñas[(reseñas["año"] == año_int) & (reseñas["recommend"] == True)]
        reseñas_positivas_por_juego = reseñas_del_año_dado["item_id"].value_counts()
        desarrolladores = 0
        for id_juego in reseñas_positivas_por_juego.index:
            try:
                desarrollador = juegos[juegos["item_id"] == id_juego]["developer"].iloc[0]
                lista_retorno.append(desarrollador)
                desarrolladores += 1
                if desarrolladores == 10: # Por las dudas dejo el "top 10" desarrolladores, aunque en realidad me voy a quedar con los primeros 3 unicos.
                    break
            except Exception as e:
                continue
        lista_desarrolladores = list(dict.fromkeys(lista_retorno))
        lista_retorno = []
        desarrollador_top_n = 1
        for desarrollador in lista_desarrolladores[:3]:
            diccionario_temporal = {}
            diccionario_temporal[f"Puesto nº {desarrollador_top_n}"] = desarrollador
            desarrollador_top_n += 1
            lista_retorno.append(diccionario_temporal)
        return lista_retorno
    except Exception as e:
        return e
    
mejor_desarrollador_del_año(2013)

[{'Puesto nº 1': 'Valve'},
 {'Puesto nº 2': 'Facepunch Studios'},
 {'Puesto nº 3': 'Re-Logic'}]

La función ya cumple con lo pedido, le voy a hacer algunas modificaciones para que quede mejor.

Septimo prototipo del endpoint

In [34]:
def mejor_desarrollador_del_año(año):
    """
    En este septimo prototipo, voy a ingresar un año a la función, y me va a devolver una lista de diccionarios con los top 3 desarrolladores
    """
    try:
        lista_retorno = [] # Creación de una lista que va almacenar los datos de salida de la función

        año_int = int(año) # Chequeo si lo ingresado a la función es un número transformable a entero, en caso de que lo sea, lo almaceno en la variable año_int

        reseñas_del_año_dado = reseñas[(reseñas["año"] == año_int) & (reseñas["recommend"] == True)] # Filtro el dataframe de reseñas con el año dado y con recommend == True

        reseñas_positivas_por_juego = reseñas_del_año_dado["item_id"].value_counts().nlargest(12) # Cuento las reseñas positivas de cada juego y se guardan unicamente los 12 juegos con más reseñas

        lista_desarrolladores = [] # Se crea una lista dónde se van a almacenar los 12 desarrolladores correspondientes con los 12 juegos con más reseñas (puede tener repetidos).

        for id_juego in reseñas_positivas_por_juego.index: # Por cada juego almacenado en la variable de reseñas positivas, se realiza una iteración
            try:
                desarrollador = juegos[juegos["item_id"] == id_juego]["developer"].iloc[0] # Se intenta almacenar en la variable desarrollador el nombre del desarrollador correspondiente al juego con las reseñas positivas

                lista_desarrolladores.append(desarrollador) # En caso de que se pueda, se agrega a la lista de desarrolladores

            except Exception as e:
                continue

        lista_desarrolladores = list(dict.fromkeys(lista_desarrolladores)) # Se dejan sólo los valores únicos
        
        desarrollador_top_n = 1 # Se crea una variable que va servir para la creación del diccionario contenido dentro de la lista de salida

        for desarrollador in lista_desarrolladores[:3]: # Por cada desarrollador del top 3 de desarrolladores únicos...

            diccionario_temporal = {} # Se crea un diccionario temporal 
            diccionario_temporal[f"Puesto nº {desarrollador_top_n}"] = desarrollador # Que va a tener cómo clave el puesto del desarrollador, empezando por el 1 y terminando en el 3
            desarrollador_top_n += 1 # Se aumenta la variable creada previamente en una unidad
            lista_retorno.append(diccionario_temporal) # Se agrega a la lista de salida el diccionario creado previamente
        return lista_retorno
    except Exception as e:
        return e
    
mejor_desarrollador_del_año(2013) # Me devuelve lo mismo, pero de forma mucho más prolija

[{'Puesto nº 1': 'Valve'},
 {'Puesto nº 2': 'Facepunch Studios'},
 {'Puesto nº 3': 'Re-Logic'}]

Me gusta cómo desarrollé esta función, pero creo que podría optimizarla más

In [36]:
reseñas

,item_id,recommend,review,año
0,1250,True,Simple yet with great replayability. In my opi...,2011
1,22200,True,It's unique and worth a playthrough.,2011
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,251610,True,I know what you think when you see this title ...,2014
4,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...
59252,730,True,its FUNNNNNNNN,2015
59255,253980,True,Awesome fantasy game if you don't mind the gra...,2015
59265,730,True,Prettyy Mad Game,2015
59267,730,True,AMAZING GAME 10/10,2015


In [44]:
# Lo primero que voy a hacer es un merge entre la tabla juegos y la de reseñas, para tener toda la data en una misma tabla.
desarrolladores_y_reseñas = pd.merge(juegos,reseñas,how="right",on="item_id")
desarrolladores_y_reseñas

,item_id,developer,recommend,review,año
0,1250,Tripwire Interactive,True,Simple yet with great replayability. In my opi...,2011
1,22200,ACE Team,True,It's unique and worth a playthrough.,2011
2,43110,NaN,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,251610,NaN,True,I know what you think when you see this title ...,2014
4,227300,SCS Software,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
49181,730,Valve,True,its FUNNNNNNNN,2015
49182,253980,Starbreeze,True,Awesome fantasy game if you don't mind the gra...,2015
49183,730,Valve,True,Prettyy Mad Game,2015
49184,730,Valve,True,AMAZING GAME 10/10,2015


In [45]:
desarrolladores_y_reseñas.isnull().sum()

item_id         0
developer    8674
recommend       0
review          0
año             0
dtype: int64

Desgraciadamente, al no saber el nombre de las desarrolladoras correspondientes a los juegos que tuvieron esas 8674 reseñas, no me aporta una información útil, por lo cuál, decido eliminar estas filas. 

In [48]:
desarrolladores_y_reseñas.dropna(inplace=True)
desarrolladores_y_reseñas

,item_id,developer,recommend,review,año
0,1250,Tripwire Interactive,True,Simple yet with great replayability. In my opi...,2011
1,22200,ACE Team,True,It's unique and worth a playthrough.,2011
4,227300,SCS Software,True,For a simple (it's actually not all that simpl...,2013
5,239030,3909,True,Very fun little game to play when your bored o...,2013
6,370360,Zachtronics,True,"""Run for fun? What the hell kind of fun is that?""",2015
...,...,...,...,...,...
49180,730,Valve,True,Neat,2015
49181,730,Valve,True,its FUNNNNNNNN,2015
49182,253980,Starbreeze,True,Awesome fantasy game if you don't mind the gra...,2015
49183,730,Valve,True,Prettyy Mad Game,2015


También lo que voy a hacer es dejar unicamente las reseñas dónde la columna "recommend" sea igual a True

In [54]:
desarrolladores_y_reseñas = desarrolladores_y_reseñas[desarrolladores_y_reseñas["recommend"] == True]
desarrolladores_y_reseñas

,item_id,developer,recommend,review,año
0,1250,Tripwire Interactive,True,Simple yet with great replayability. In my opi...,2011
1,22200,ACE Team,True,It's unique and worth a playthrough.,2011
4,227300,SCS Software,True,For a simple (it's actually not all that simpl...,2013
5,239030,3909,True,Very fun little game to play when your bored o...,2013
6,370360,Zachtronics,True,"""Run for fun? What the hell kind of fun is that?""",2015
...,...,...,...,...,...
49180,730,Valve,True,Neat,2015
49181,730,Valve,True,its FUNNNNNNNN,2015
49182,253980,Starbreeze,True,Awesome fantasy game if you don't mind the gra...,2015
49183,730,Valve,True,Prettyy Mad Game,2015


Bueno, a partir de esta tabla, ya puedo plantear otro prototipo del cuarto endpoint.

Octavo prototipo del endpoint

In [57]:
def mejor_desarrollador_del_año(año):
    """
    En este octavo prototipo, voy a ingresar un año a la función, y me va a devolver una lista de diccionarios con los top 3 desarrolladores
    """
    try:
        lista_retorno = [] # Creación de una lista que va almacenar los datos de salida de la función

        año_int = int(año) # Chequeo si lo ingresado a la función es un número transformable a entero, en caso de que lo sea, lo almaceno en la variable año_int

        reseñas_del_año_dado = desarrolladores_y_reseñas[desarrolladores_y_reseñas["año"] == año_int] # Filtro el dataframe de reseñas con el año dado y con recommend == True

        desarrolladores_top = reseñas_del_año_dado["developer"].value_counts().nlargest(3) # Cuento las reseñas positivas de cada desarrollador y se guardan unicamente los 3 desarrolladores con más reseñas

        puesto = 1

        for desarrollador in desarrolladores_top.index: # Por cada desarrollador del top 3 de desarrolladores únicos...

            diccionario_temporal = {} # Se crea un diccionario temporal 
            diccionario_temporal[f"Puesto nº {puesto}"] = desarrollador # Que va a tener cómo clave el puesto del desarrollador, empezando por el 1 y terminando en el 3
            puesto += 1 # Se aumenta la variable creada previamente en una unidad
            lista_retorno.append(diccionario_temporal) # Se agrega a la lista de salida el diccionario creado previamente
        return lista_retorno
    except Exception as e:
        return e
    
mejor_desarrollador_del_año(2013) # Me devuelve lo mismo, pero de forma optimizada

[{'Puesto nº 1': 'Valve'},
 {'Puesto nº 2': 'Facepunch Studios'},
 {'Puesto nº 3': 'Re-Logic'}]

Bueno, ya tengo el resultado de la función, pero ahora me gustaría tenga una forma así:

{Año : {
    Puesto nº1 : {
        Desarrollador X : 175343 recomendaciones
    },
    Puesto nº2 : {
        Desarrollador Y : 75342 recomendaciones
    },
    Puesto nº3 : {
        Desarrollador Z : 1753 recomendaciones
    }
}
}

Así que a eso voy

Noveno prototipo del endpoint

In [7]:
import pandas as pd
def mejor_desarrollador_del_año(año):
    """
    En este noveno prototipo, voy a ingresar un año a la función, y me va a devolver una lista de diccionarios con los top 3 desarrolladores
    """
    try:
        desarrolladores_y_reseñas = pd.read_parquet("../Datasets/endpoint_4/desarrolladores_y_reseñas.parquet")

        diccionario_salida = {}

        año_int = int(año) # Chequeo si lo ingresado a la función es un número transformable a entero, en caso de que lo sea, lo almaceno en la variable año_int

        reseñas_del_año_dado = desarrolladores_y_reseñas[desarrolladores_y_reseñas["año"] == año_int] # Filtro el dataframe de reseñas con el año dado y con recommend == True

        desarrolladores_top = reseñas_del_año_dado["developer"].value_counts().nlargest(3) # Cuento las reseñas positivas de cada desarrollador y se guardan unicamente los 3 desarrolladores con más reseñas

        diccionario_interno = {f"Puesto nº{puesto}": {desarrollador: f"{recomendaciones} recomendaciones"} 
                               for puesto, (desarrollador, recomendaciones) in enumerate(desarrolladores_top.items(), 1)}

        diccionario_salida[año_int] = diccionario_interno # Finalmente el diccionario de salida tiene cómo clave el año ingresado y cómo valor el diccionario interno previamente creado

        return diccionario_salida
    except Exception as e:
        return e
    
mejor_desarrollador_del_año(2015) # Me devuelve lo mismo, pero de forma optimizada

{2015: {'Puesto nº1': {'Valve': '2573 recomendaciones'},
  'Puesto nº2': {'Facepunch Studios': '533 recomendaciones'},
  'Puesto nº3': {'Smartly Dressed Games': '284 recomendaciones'}}}

In [83]:
desarrolladores_y_reseñas.to_parquet("../Datasets/endpoint_4/desarrolladores_y_reseñas.parquet",compression="snappy")